In [33]:

import pandas as pd
# from PIL import Image
import cv2
import numpy as np



class driveFile:
  def __init__(self):
    self.path=""
  # def __init__(self, mountpath):
  #   import os
  #   self.path = mountpath
  #   from google.colab import drive
  #   drive.mount(self.path, force_remount=True)
  def dataload(self,folder_path):
    img=[]
    label=[]
    import os
    self.path1=os.path.join(self.path,folder_path)
    
    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    img_array=np.array(img)
    return img_array

In [39]:
# drive=driveFile('/content/drive')
# images,labels=drive.dataload('MyDrive/train_set')
drive=driveFile()
images,labels=drive.dataload('train_set')

Skipping invalid path or .DS_Store: train_set/FU-athlete-foot
Skipping invalid path or .DS_Store: train_set/BA- cellulitis


In [50]:

class train:
  def __init__(self,X,w,b,Y,learning_rate):
    self.X=X
    self.w=w
    self.b=b
    self.y=Y
    self.learning_rate=learning_rate
    
    self.binary_cross_entropy()
    self.backward_prop()

  
  def perceptron(self):
      y_linearop=np.dot(self.X,self.w)+self.b
      #unbounded
      return y_linearop

  def sigmoid(self):#activatn fn
      y_predprob=1/(1+np.exp(-self.perceptron()))
      self.y_pred=y_predprob


  def binary_cross_entropy(self):
      self.sigmoid()
      self.y_pred = np.clip(self.y_pred, 1e-10, 1 - 1e-10)  # Prevent extreme values
      self.loss=-np.mean((self.y*np.log(self.y_pred))+((1-self.y)*np.log(1-self.y_pred)))
      print(self.loss)

  def backward_prop(self):
    dloss_dy_pred=-(self.y/self.y_pred)+((1-self.y)/(1-self.y_pred))
    dsigm_dlinearoutput=self.y_pred*(1-self.y_pred)
    #bcz dsigmoid/dz=sigmod-sigmoid^2
    dloss_dlinearoutput=dloss_dy_pred*dsigm_dlinearoutput
    # (f(g(x)))`=f`(g(x))g'(x)

    self.dw = np.dot(self.X.T, dloss_dlinearoutput) / self.X.shape[0]
    self.db = np.mean(dloss_dlinearoutput)
    self.update_para()


  def update_para(self):
    self.w -= self.learning_rate * self.dw
    self.b -= self.learning_rate * self.db

In [48]:
print(images.shape)
print(labels.shape)
images=np.array([img.flatten() for img in images])
# images=np.array([img.reshape(-1,1) for img in images])
print(images.shape)
labels=np.array([1 if label=='BA- cellulitis' else 0 for label in labels])
# ba-cellulitis 1, foot 0


# # imgs=[img1,
# # img2,
# # img3,
# #
# #
# # img200]



(197, 400)
(197,)
(197, 400)


1D Array (Shape (400,)):
In a 1D array, NumPy doesn't explicitly distinguish between rows and columns. It's just a sequence of 400 elements along a single axis (the x-axis). When you perform element-wise operations, such as scalar multiplication, it operates directly on each element because NumPy sees it as a single dimension (or vector). 2D Array (Shape (400, 1)):
Here, NumPy treats it as a column vector. The shape (400, 1) has two axes: the first axis (rows) has 400 elements, and the second axis (columns) has 1 element. Even though there is only one column, it's a 2D structure. When performing element-wise operations, NumPy knows it needs to perform operations on a 2D grid (or matrix).

In [51]:

w=np.random.randn(images.shape[1],)*0.01
b=0
learning_rate=0.5
train=train(images,w,b,labels,learning_rate)

0.697756724586146
